# Generics
자바는 역사적으로 제너릭이 없었던 시절이 있고, 하위호환성을 중요시하는 언어일 수밖에 없다보니
제너릭으로 선언한 클래스도 제너릭이 아닌 시절의 클래스처럼 작동하게 할 수 있음.

제너릭 `class C<T> { ... }`이런 모든 참조 타입 T에 대해서 활용할 수 있는 클래스를
제너릭이 아닌 것처럼 `new C()` 이렇게 해서 사용할 수 있다는 끔찍한 사실에 주의해야 합니다.
이렇게 사용하면 `new C<Object>()`로 동작. 그래서 에러가 안나는데 현실적으로 이런 동작은
아무도 원하지 않으므로 자바에서 제너릭 클래스를 쓸 대는 `<`와 `>`를 절대 생략하지 말자!!!!

## The problem
Let suppose we have a code somewhere to extract credentials from a config file
Note: this code generates two warnings, that we will discuss later.


In [1]:
record Pair(Object first, Object second) { }
List getCredentials() {
  List list = new ArrayList();
  list.add(new Pair("login", "admin"));
  list.add(new Pair("password", "password"));
  return list; 
}
System.out.println(getCredentials());


[Pair[first=login, second=admin], Pair[first=password, second=password]]


And in another code, we want to use the method `getCredential()`, we may write
a code like this


In [2]:
List list = getCredentials();
String value = list.get(0).toString();

This code will compile but fail at runtime, it throws a `ClassCastException`
because in `getCredential()`, we are creating a list of pairs but in the code
above, we try to extract the first value from the list as a String


Here, we have lost the fact that the list returned by `getCredential()`
is a list of pairs.
Generics allows to propagate the type of values stored inside a class or
more generally, allows to declare relations between types


We have said above that the code generate warnings, it's because java.util.List
is declared as a generics class in the JDK API. So the compiler let you use
List as type if you interact with a code written before generics were introduced
to Java (2004) but emits a warning saying you should not declare it that way.  


## Generics
제너릭을 활용해 선언할 수 있는 것이 두 가지 있다
- 제너릭 클래스(parameterized class 또는 genric class)
- 제너릭 메소드(parameterized method 또는 generic method)

## Parameterized class 또는 generic class
A parameterized class is a class that declares type variables
(variable that contains a type) and use them whenever we can use a type.


### Declaration
The type variables are declared after the name of the class.


In [3]:
record Pair<F, S>(F first, S second) { }


When used, as a user of the generics you have to specify the type
of each type variable (here F is String and S is Integer)
so pair.first() which is typed as F will return a String and
pair.second() which is typed as S will return an int.


In [4]:
var pair = new Pair<String, Integer>("port", 8080);
String first = pair.first();
int second = pair.second();


If you don't understand why in between the '<' and '>',
there is a Integer here and not an int
Don't worry, it's explained in next chapter


So the idea of a generics class is to specify the type arguments `<String, Integer>`
when you use it and the compiler will propagate the types.


### static context
You may have notice that two different instances may have different type arguments


In [5]:
var pair1 = new Pair<String, Integer>("port", 8080);
var pair2 = new Pair<String, String>("captain amarica", "shield");


So the type variable (F and S) are not available inside a static method of Pair.
A static method is called on the class `Pair.hello()`, not on an instance.



In [6]:
record Pair<F, S>(F first, S second) {
  static void hello() {
    // can not access F and S here !
    new Integer(3);
    // new F(); // 에러
  }
}


제너릭 클래스 필드/메소드도 제너릭이 아닌 클래스 메소드와 마찬가지로
꺾쇠 없이 바로 클래스 이름 다음에 점을 찍고 쓴다

In [7]:
Pair.hello()

In [8]:
Pair<String,Integer>.hello()

CompilerException: 

## Parameterized methods 또는 generic method
Like a class, a method can be parameterized, by declaring the type variables
in between `<` and `>` before the return type
So instead of


In [9]:
Object chooseOne(Object o1, Object o2) {
  var random = ThreadLocalRandom.current();
  return random.nextBoolean()? o1: o2;
}

String s = chooseOne("day", "night").toString();

System.out.println(s)

day


We can write


In [10]:
<T> T chooseOne(T o1, T o2) {
  var random = ThreadLocalRandom.current();
  return random.nextBoolean()? o1: o2;
}

// 제너릭 클래스와 달리 제너릭 메소드는 꺾쇠를 생략 (컴파일러가 알아서 유추)
String s = chooseOne("day", "night");

System.out.println(s);


night


## Inference
So, we can now rewrite `getCredentials()`, to correctly type it


In [11]:
List<Pair<String, String>> getCredentials() {
  // List<Pair<String, String>> list = new ArrayList<Pair<String, String>>();
  var list = new ArrayList<Pair<String, String>>();
  // list.add(new Pair<String, String>("login", "admin"));
  // list.add(new Pair<String, String>("password", "password"));
  list.add(new Pair<>("login", "admin"));
  list.add(new Pair<>("password", "password"));
  return list; 
}
System.out.println(getCredentials());


[Pair[first=login, second=admin], Pair[first=password, second=password]]


but it's quite verbose, so in Java, we have a mechanism called __inference__
to let the compiler try to guess itself the type arguments instead of
having to specify them by hand


### inference of variable local type
The keyword `var` ask the compiler to find the type of the left of `=`
from the type of the right of `=`.
So instead of


In [12]:
List<Pair<String, String>> list = new ArrayList<Pair<String, String>>();
System.out.println(list);


[]


using `var` we get


In [13]:
var list = new ArrayList<Pair<String, String>>();
System.out.println(list);


[]


### inference when using `new`
You can ask the compiler to find the type using the left type and the arguments
using the diamond syntax `<>`.
So instead of


In [14]:
Pair<String, String> pair = new Pair<String, String>("login", "admin");


using the diamond syntax `<>`


In [15]:
Pair<String, String> pair = new Pair<>("login", "admin");
System.out.println(pair);


Pair[first=login, second=admin]


The left type can be also found when you do a `return` 


In [16]:
Pair<String, String> getOnePair() {
  return new Pair<>("login", "admin");
}
System.out.println(getOnePair());


Pair[first=login, second=admin]


or using the type of the parameter of the method


In [17]:
var list = new ArrayList<Pair<String, String>>();
list.add(new Pair<>("login", "admin"));


true

You may wonder what if we are using `var` and the diamond `<>` at the same time
When the inference doesn't known, it using `Object`


In [18]:
var objectList = new ArrayList<>();   // this is a list of Object


### inference of parameterized method
Type arguments of a parameterized method are inferred by default and we have
to use a special syntax if we want to specify the type arguments


That's why when we have


In [19]:
class Utils {
  static <T> T chooseOne(T o1, T o2) {
    var random = ThreadLocalRandom.current();
    return random.nextBoolean()? o1: o2;
  }
}


we can write


In [20]:
System.out.println(Utils.chooseOne("foo", "bar"));


bar


and if we want to specify the type arguments, you have to
specify them in between `<` and `>`, after the `.` and
before the name of the method


In [21]:
System.out.println(Utils.<String>chooseOne("foo", "bar"));


foo


### Raw type
Types without the '<' '>', raw types in Java speak, are still supported
to interact with old codes so you may by mistake forget the '<' '>' and
have the declaration to compile.
But it will be nasty when trying to use such type.


The for-loop below doesn't compile because StringList is an AbstractList
so a List of Object and not a List<String>


In [22]:
class StringList extends AbstractList {  // should be AbstractList<String>
  public int size() {
    return 5;
  }
  public String get(int index) {
    Objects.checkIndex(index, 5);
    return "" + index;
  }
}

for(String s: new StringList()) { }

CompilerException: 

### So using inference
`getCredentials()` can be simplified to


In [23]:
List<Pair<String, String>> getCredentials() {
  var list = new ArrayList<Pair<String, String>>();
  list.add(new Pair<>("login", "admin"));
  list.add(new Pair<>("password", "password"));
  return list; 
}
var list = getCredentials();
/*
 String value = (String) list.get(0);
*/


And the last line (commented) that was throwing a ClassCastException
now does not compile thank to the use of generics.


## Bounds
By default a type variable like `T` acts as Object, i.e.
you can call on T only the public methods of java.lang.Object.
you can defines a more precise __bound__ reusing the keyword `extends`
Note: `extends` in this context mean subtype not inherits from.


In [24]:
// 레코드라도 compareTo 같은 크기 비교 메소드를 자동으로 만들어주지는 않음 
new Pair<>(1,2).compareTo(new Pair<>(3,4))

CompilerException: 

In [25]:
"def".compareTo("abc") // 정수, 실수, 문자열 등은 compareTo 지원

3

In [26]:
"def" instanceof Comparable

true

In [27]:
<T extends Comparable<T>> T min(T o1, T o2) {
  return (o1.compareTo(o2) < 0)? o1: o2;
}

System.out.println(min("day", "night"));

day


Because the bound of T is an object, you can not declare a List<int> !
But you can declare a List<Integer> instead, see the next chapter for that !
